In [1]:
import pandas as pd
import numpy as np

In [2]:
file_list = ['./110S1/', './110S2/', './110S3/', './110S4/', './111S1/', './111S2/', './111S3/', './111S4/']
dfs = []

for file in file_list:
    df_temp = pd.read_csv(file + 'A_lvr_land_A.csv')
    dfs.append(df_temp)

# 將資料串接起來
dfA = pd.concat(dfs)

In [3]:
dfA

,鄉鎮市區,交易標的,土地位置建物門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,移轉層次,...,車位類別,車位移轉總面積(平方公尺),車位總價元,備註,編號,主建物面積,附屬建物面積,陽台面積,電梯,移轉編號
0,The villages and towns urban district,transaction sign,land sector position building sector house num...,land shifting total area square meter,the use zoning or compiles and checks,the non-metropolis land use district,non-metropolis land use,transaction year month and day,transaction pen number,shifting level,...,the berth category,berth shifting total area square meter,the berth total price NTD,the note,serial number,NaN,NaN,NaN,NaN,NaN
1,文山區,房地(土地+建物),興隆路二段２１７巷８之７號五樓,8.65,都市：其他:第三種住宅區。,NaN,NaN,1091029,土地1建物1車位0,五層,...,NaN,0.0,0,陽台外推；其他增建；,RPWOMLKKJIPFFAA87DA,25.66,0.7,5.79,有,NaN
2,文山區,房地(土地+建物),和平東路四段６５巷７７弄１號,86.0,都市：其他:第二種住宅區。,NaN,NaN,1091105,土地1建物1車位0,全,...,NaN,0.0,0,陽台外推；其他增建；,RPVPMLSKJIPFFAA68DA,133.66,35.95,6.11,無,NaN
3,文山區,建物,羅斯福路六段２６巷２號,0.0,NaN,NaN,NaN,1091102,土地0建物1車位0,一層,...,NaN,0.0,0,NaN,RPSOMLOLJIPFFAA97DA,79.57,0.0,0.0,無,NaN
4,文山區,房地(土地+建物),興隆路四段１４５巷３６號四樓,33.62,都市：其他:第三種住宅區。,NaN,NaN,1091123,土地1建物1車位0,四層,...,NaN,0.0,0,頂樓加蓋；,RPSNMLKMJIPFFAA57DA,89.45,0.0,16.76,無,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5410,士林區,房地(土地+建物),臺北市士林區文林路１４４巷２０號四樓,10.66,商,NaN,NaN,1040309,土地1建物1車位0,四層,...,NaN,0.0,0,NaN,RPXOMLKMOHHGFEA28DA,25.22,1.91,0.0,有,NaN
5411,士林區,房地(土地+建物),臺北市士林區文林路１４４巷１０號四樓,11.07,商,NaN,NaN,1030829,土地1建物1車位0,四層,...,NaN,0.0,0,NaN,RPWNMLKMOHHGFEA08DA,25.49,0.0,2.68,有,NaN
5412,士林區,房地(土地+建物),臺北市士林區文林路１４４巷１６號,10.66,商,NaN,NaN,1030520,土地1建物1車位0,一層,...,NaN,0.0,0,NaN,RPXNMLKMOHHGFEA18DA,25.22,1.91,0.0,有,NaN
5413,士林區,房地(土地+建物),臺北市士林區文林路１４４巷１０號三樓,11.07,商,NaN,NaN,1030430,土地1建物1車位0,三層,...,NaN,0.0,0,NaN,RPVNMLKMOHHGFEA97DA,25.49,0.0,2.68,有,NaN


In [4]:
dfsb = []

for file in file_list:
    df_temp = pd.read_csv(file + 'A_lvr_land_B.csv')
    dfsb.append(df_temp)

# 將資料串接起來
dfB = pd.concat(dfsb)

In [5]:
dfB

,鄉鎮市區,交易標的,土地位置建物門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,移轉層次,...,有無管理組織,總價元,單價元平方公尺,車位類別,車位移轉總面積平方公尺,車位總價元,備註,編號,建案名稱,棟及號
0,The villages and towns urban district,transaction sign,land sector position building sector house num...,land shifting total area square meter,the use zoning or compiles and checks,the non-metropolis land use district,non-metropolis land use,transaction year month and day,transaction pen number,shifting level,...,Whether there is manages the organization,total price NTD,the unit price (NTD / square meter),the berth category,berth shifting total area square meter,the berth total price NTD,the note,serial number,NaN,NaN
1,中正區,房地(土地+建物),臺北市中正區濟南路一段13巷6號4樓之2,9.23,住,NaN,NaN,1090311,土地1建物1車位0,四層,...,無,12900000,344920,NaN,0.0,0,預售屋雨遮不計價,RPWNMLLKKIPFFBA08CB,NaN,NaN
2,中正區,房地(土地+建物)+車位,臺北市中正區濟南路一段13巷6號3樓之2,10.66,住,NaN,NaN,1090721,土地1建物1車位1,三層,...,無,15170000,356150,升降平面,45.53,1850000,預售屋雨遮不計價,RPXNMLLKKIPFFBA18CB,NaN,NaN
3,中正區,房地(土地+建物),臺北市中正區濟南路一段13巷6號3樓之1,9.23,住,NaN,NaN,1090621,土地1建物1車位0,三層,...,無,13750000,367647,NaN,0.0,0,預售屋雨遮不計價,RPPOMLLKKIPFFBA47CB,NaN,NaN
4,中正區,房地(土地+建物),臺北市中正區濟南路一段13巷6號4樓之5,9.0,住,NaN,NaN,1090802,土地1建物1車位0,四層,...,無,13300000,363786,NaN,0.0,0,預售屋雨遮不計價,RPROMLLKKIPFFBA67CB,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,松山區,房地(土地+建物),臺北市松山區八德路四段599號,5.64,商,NaN,NaN,1111107,土地5建物1車位0,四層,...,無,16500000,284434,NaN,0.0,0,NaN,RPQNMLPJKIHGFDA47EB,松捷樂,A-4F號
642,大安區,房地(土地+建物),臺北市大安區羅斯福路三段283巷21弄7號,14.79,住,NaN,NaN,1111108,土地1建物1車位0,四層,...,無,28080000,387364,NaN,0.0,0,NaN,RPPNMLSJKIHGFFA67CB,大安得逸,A號
643,北投區,房地(土地+建物)+車位,臺北市北投區文林北路與文承路交叉口,32.25,住,NaN,NaN,1111109,土地3建物1車位2,三層,...,無,64860000,354202,坡道平面,55.21,6700000,交易總價含下列非屬預售屋價格之費用:親友、員工、共有人或其他特殊關係間之交易；,RPPNMLSJKIHGFEA57CB,華固上文林,A5-03號
644,大安區,房地(土地+建物),臺北市大安區通化街38巷,9.81,商,NaN,NaN,1111109,土地1建物1車位0,八層,...,無,17550000,387161,NaN,0.0,0,NaN,RPRNMLOKKIHGFFA57CB,大安富筑,A-8F號


In [6]:
# 把欄位名稱修改一致
dfA.rename(columns={'車位移轉總面積(平方公尺)': '車位移轉總面積平方公尺'}, inplace=True)

# 增加是否為預售屋的欄位
dfA['預售屋'] = '0'
dfB['預售屋'] = '1'

In [7]:
# 移除dfA和dfB的第一個row（英文名稱）
dfA = dfA.drop(dfA.index[0])
dfB = dfB.drop(dfB.index[0])

# 合併一般不動產和預售屋的資料
df_merged = pd.concat([dfA, dfB])

In [8]:
df_merged.columns

Index(['鄉鎮市區', '交易標的', '土地位置建物門牌', '土地移轉總面積平方公尺', '都市土地使用分區', '非都市土地使用分區',
       '非都市土地使用編定', '交易年月日', '交易筆棟數', '移轉層次', '總樓層數', '建物型態', '主要用途', '主要建材',
       '建築完成年月', '建物移轉總面積平方公尺', '建物現況格局-房', '建物現況格局-廳', '建物現況格局-衛',
       '建物現況格局-隔間', '有無管理組織', '總價元', '單價元平方公尺', '車位類別', '車位移轉總面積平方公尺', '車位總價元',
       '備註', '編號', '主建物面積', '附屬建物面積', '陽台面積', '電梯', '移轉編號', '預售屋', '建案名稱',
       '棟及號'],
      dtype='object')

In [9]:
df_merged.head(3)

,鄉鎮市區,交易標的,土地位置建物門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,移轉層次,...,備註,編號,主建物面積,附屬建物面積,陽台面積,電梯,移轉編號,預售屋,建案名稱,棟及號
1,文山區,房地(土地+建物),興隆路二段２１７巷８之７號五樓,8.65,都市：其他:第三種住宅區。,NaN,NaN,1091029,土地1建物1車位0,五層,...,陽台外推；其他增建；,RPWOMLKKJIPFFAA87DA,25.66,0.7,5.79,有,NaN,0,NaN,NaN
2,文山區,房地(土地+建物),和平東路四段６５巷７７弄１號,86.0,都市：其他:第二種住宅區。,NaN,NaN,1091105,土地1建物1車位0,全,...,陽台外推；其他增建；,RPVPMLSKJIPFFAA68DA,133.66,35.95,6.11,無,NaN,0,NaN,NaN
3,文山區,建物,羅斯福路六段２６巷２號,0.0,NaN,NaN,NaN,1091102,土地0建物1車位0,一層,...,NaN,RPSOMLOLJIPFFAA97DA,79.57,0.0,0.0,無,NaN,0,NaN,NaN


## Exploratory Data Analysis

In [10]:
# from matplotlib.font_manager import fontManager

# for i in sorted(fontManager.get_font_names()):
#     print(i)

In [11]:
# import matplotlib.pyplot as plt

# # plt.rcParams['font.sans-serif'] = ['SimHei']  # 替換成您安裝的中文字型

# df_merged['鄉鎮市區'].value_counts().plot(kind='bar')

# # 設定圖表標題和軸標籤
# plt.title('鄉鎮市區資料數統計')
# plt.xlabel('鄉鎮市區')
# plt.ylabel('數量')

# # 顯示圖表
# plt.show()

## Data Cleaning

In [12]:
# 移除不需要的欄位
df_merged = df_merged.drop(['交易標的', '都市土地使用分區', '非都市土地使用分區', '非都市土地使用編定', '總樓層數', '主要用途',
              '主要建材', '車位總價元', '編號', '移轉編號', '建案名稱', '棟及號'], axis=1)

In [13]:
df_merged.head(3)

,鄉鎮市區,土地位置建物門牌,土地移轉總面積平方公尺,交易年月日,交易筆棟數,移轉層次,建物型態,建築完成年月,建物移轉總面積平方公尺,建物現況格局-房,...,總價元,單價元平方公尺,車位類別,車位移轉總面積平方公尺,備註,主建物面積,附屬建物面積,陽台面積,電梯,預售屋
1,文山區,興隆路二段２１７巷８之７號五樓,8.65,1091029,土地1建物1車位0,五層,住宅大樓(11層含以上有電梯),0850715,40.989999999999995,1,...,7300000,178092,NaN,0.0,陽台外推；其他增建；,25.66,0.7,5.79,有,0
2,文山區,和平東路四段６５巷７７弄１號,86.0,1091105,土地1建物1車位0,全,透天厝,0760413,175.72,5,...,24500000,139426,NaN,0.0,陽台外推；其他增建；,133.66,35.95,6.11,無,0
3,文山區,羅斯福路六段２６巷２號,0.0,1091102,土地0建物1車位0,一層,公寓(5樓含以下無電梯),0570614,79.57,4,...,8500000,106824,NaN,0.0,NaN,79.57,0.0,0.0,無,0


In [14]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# 將建築完成年月改成建築完成幾年
df_merged['建築完成距今'] = df_merged['建築完成年月'].apply(lambda x: datetime.strptime(str(int(x) + 19110000), '%Y%m%d').date() if pd.notnull(x) and isinstance(x, (int, str)) and len(str(x)) in [6, 7, 8] else None)

# 計算交易距今的相差年數
current_date = datetime.now().date()  # 獲取當前日期
df_merged['建築完成距今'] = df_merged['建築完成距今'].apply(lambda x: relativedelta(current_date, x).years)  # 計算相差的年數

In [15]:
df_merged

,鄉鎮市區,土地位置建物門牌,土地移轉總面積平方公尺,交易年月日,交易筆棟數,移轉層次,建物型態,建築完成年月,建物移轉總面積平方公尺,建物現況格局-房,...,單價元平方公尺,車位類別,車位移轉總面積平方公尺,備註,主建物面積,附屬建物面積,陽台面積,電梯,預售屋,建築完成距今
1,文山區,興隆路二段２１７巷８之７號五樓,8.65,1091029,土地1建物1車位0,五層,住宅大樓(11層含以上有電梯),0850715,40.989999999999995,1,...,178092,NaN,0.0,陽台外推；其他增建；,25.66,0.7,5.79,有,0,26
2,文山區,和平東路四段６５巷７７弄１號,86.0,1091105,土地1建物1車位0,全,透天厝,0760413,175.72,5,...,139426,NaN,0.0,陽台外推；其他增建；,133.66,35.95,6.11,無,0,36
3,文山區,羅斯福路六段２６巷２號,0.0,1091102,土地0建物1車位0,一層,公寓(5樓含以下無電梯),0570614,79.57,4,...,106824,NaN,0.0,NaN,79.57,0.0,0.0,無,0,54
4,文山區,興隆路四段１４５巷３６號四樓,33.62,1091123,土地1建物1車位0,四層,公寓(5樓含以下無電梯),NaN,106.21,3,...,110159,NaN,0.0,頂樓加蓋；,89.45,0.0,16.76,無,0,0
5,文山區,興隆路三段１９２巷２弄２９號八樓,9.13,1070309,土地1建物1車位0,八層,華廈(10層含以下有電梯),1090825,44.01,1,...,204499,NaN,0.0,NaN,25.66,2.13,0.0,有,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,松山區,臺北市松山區八德路四段599號,5.64,1111107,土地5建物1車位0,四層,其他,NaN,58.01,0,...,284434,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,0
642,大安區,臺北市大安區羅斯福路三段283巷21弄7號,14.79,1111108,土地1建物1車位0,四層,華廈(10層含以下有電梯),NaN,72.49,2,...,387364,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,0
643,北投區,臺北市北投區文林北路與文承路交叉口,32.25,1111109,土地3建物1車位2,三層,住宅大樓(11層含以上有電梯),NaN,219.41,3,...,354202,坡道平面,55.21,交易總價含下列非屬預售屋價格之費用:親友、員工、共有人或其他特殊關係間之交易；,NaN,NaN,NaN,NaN,1,0
644,大安區,臺北市大安區通化街38巷,9.81,1111109,土地1建物1車位0,八層,華廈(10層含以下有電梯),NaN,45.33,1,...,387161,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,0


In [16]:
# 計算每個鄉鎮市區的建築完成距今的平均值
average_by_district = df_merged.groupby('鄉鎮市區')['建築完成距今'].mean()

# 依據鄉鎮市區填入建築完成距今為0的資料的平均值
df_merged['建築完成距今'] = df_merged.apply(lambda row: average_by_district[row['鄉鎮市區']] if row['建築完成距今'] == 0 else row['建築完成距今'], axis=1)
df_merged.loc[df_merged['預售屋'] == '1', '建築完成距今'] = 0

In [17]:
df_merged.drop('建築完成年月', axis=1, inplace=True)

In [18]:
df_merged.head(3)

,鄉鎮市區,土地位置建物門牌,土地移轉總面積平方公尺,交易年月日,交易筆棟數,移轉層次,建物型態,建物移轉總面積平方公尺,建物現況格局-房,建物現況格局-廳,...,單價元平方公尺,車位類別,車位移轉總面積平方公尺,備註,主建物面積,附屬建物面積,陽台面積,電梯,預售屋,建築完成距今
1,文山區,興隆路二段２１７巷８之７號五樓,8.65,1091029,土地1建物1車位0,五層,住宅大樓(11層含以上有電梯),40.989999999999995,1,1,...,178092,NaN,0.0,陽台外推；其他增建；,25.66,0.7,5.79,有,0,26.0
2,文山區,和平東路四段６５巷７７弄１號,86.0,1091105,土地1建物1車位0,全,透天厝,175.72,5,4,...,139426,NaN,0.0,陽台外推；其他增建；,133.66,35.95,6.11,無,0,36.0
3,文山區,羅斯福路六段２６巷２號,0.0,1091102,土地0建物1車位0,一層,公寓(5樓含以下無電梯),79.57,4,1,...,106824,NaN,0.0,NaN,79.57,0.0,0.0,無,0,54.0


In [19]:
# 把鄉鎮市區變成dummy variables
df_merged = pd.get_dummies(df_merged, columns=['鄉鎮市區'])
df_merged.head(3)

,土地位置建物門牌,土地移轉總面積平方公尺,交易年月日,交易筆棟數,移轉層次,建物型態,建物移轉總面積平方公尺,建物現況格局-房,建物現況格局-廳,建物現況格局-衛,...,鄉鎮市區_信義區,鄉鎮市區_內湖區,鄉鎮市區_北投區,鄉鎮市區_南港區,鄉鎮市區_士林區,鄉鎮市區_大同區,鄉鎮市區_大安區,鄉鎮市區_文山區,鄉鎮市區_松山區,鄉鎮市區_萬華區
1,興隆路二段２１７巷８之７號五樓,8.65,1091029,土地1建物1車位0,五層,住宅大樓(11層含以上有電梯),40.989999999999995,1,1,1,...,0,0,0,0,0,0,0,1,0,0
2,和平東路四段６５巷７７弄１號,86.0,1091105,土地1建物1車位0,全,透天厝,175.72,5,4,3,...,0,0,0,0,0,0,0,1,0,0
3,羅斯福路六段２６巷２號,0.0,1091102,土地0建物1車位0,一層,公寓(5樓含以下無電梯),79.57,4,1,1,...,0,0,0,0,0,0,0,1,0,0


In [20]:
# 將"交易筆棟數"拆成3個欄位
df_merged[['交易土地', '交易建物', '交易車位']] = df_merged['交易筆棟數'].str.extract(r'土地(\d+)建物(\d+)車位(\d+)')

# 將數字轉換為整數型態
df_merged[['交易土地', '交易建物', '交易車位']] = df_merged[['交易土地', '交易建物', '交易車位']].astype(int)

# 移除原始的「交易筆棟數」欄位
df_merged.drop('交易筆棟數', axis=1, inplace=True)

In [21]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# 將交易年月日改成交易完成幾年
df_merged['交易年月日'] = df_merged['交易年月日'].apply(lambda x: datetime.strptime(str(int(x)+19110000), '%Y%m%d'))  # 將字串轉換為日期格式

# 計算交易距今的相差年數
current_date = datetime.now().date()  # 獲取當前日期
df_merged['交易距今'] = df_merged['交易年月日'].apply(lambda x: relativedelta(current_date, x).years)  # 計算相差的年數

# 移除原始的「交易年月日」欄位
df_merged.drop('交易年月日', axis=1, inplace=True)

In [22]:
df_merged.head(3)

,土地位置建物門牌,土地移轉總面積平方公尺,移轉層次,建物型態,建物移轉總面積平方公尺,建物現況格局-房,建物現況格局-廳,建物現況格局-衛,建物現況格局-隔間,有無管理組織,...,鄉鎮市區_士林區,鄉鎮市區_大同區,鄉鎮市區_大安區,鄉鎮市區_文山區,鄉鎮市區_松山區,鄉鎮市區_萬華區,交易土地,交易建物,交易車位,交易距今
1,興隆路二段２１７巷８之７號五樓,8.65,五層,住宅大樓(11層含以上有電梯),40.989999999999995,1,1,1,有,有,...,0,0,0,1,0,0,1,1,0,2
2,和平東路四段６５巷７７弄１號,86.0,全,透天厝,175.72,5,4,3,有,有,...,0,0,0,1,0,0,1,1,0,2
3,羅斯福路六段２６巷２號,0.0,一層,公寓(5樓含以下無電梯),79.57,4,1,1,有,無,...,0,0,0,1,0,0,0,1,0,2


In [23]:
import re

def extract_street(address):
    # 定義街道名稱的正則表達式
    pattern = r"(?P<city>[\u4e00-\u9fa5]+市)?(?P<district>[\u4e00-\u9fa5]+[區市里])?(?P<street>[\u4e00-\u9fa5]+[路街巷道段]).*"

    # 使用正則表達式進行匹配
    match = re.match(pattern, address)
    
    if match:
        street_name = match.group("street")
        
        # 去除市區、巷、弄、號等元素
        street_name = re.sub(r"[市區里].*$", "", street_name)
        street_name = re.sub(r"[巷弄號].*$", "", street_name)
        return street_name
    else:
        return None

df_merged['路段'] = df_merged['土地位置建物門牌'].apply(extract_street)

In [24]:
from sklearn.preprocessing import LabelEncoder

# 創建 LabelEncoder 物件
label_encoder = LabelEncoder()

# 將類別變數進行編碼
df_merged['路段'] = label_encoder.fit_transform(df_merged['路段'])

# 刪除土地位置建物門牌
df_merged.drop('土地位置建物門牌', axis=1, inplace=True)

In [25]:
df_merged.head(3)

,土地移轉總面積平方公尺,移轉層次,建物型態,建物移轉總面積平方公尺,建物現況格局-房,建物現況格局-廳,建物現況格局-衛,建物現況格局-隔間,有無管理組織,總價元,...,鄉鎮市區_大同區,鄉鎮市區_大安區,鄉鎮市區_文山區,鄉鎮市區_松山區,鄉鎮市區_萬華區,交易土地,交易建物,交易車位,交易距今,路段
1,8.65,五層,住宅大樓(11層含以上有電梯),40.989999999999995,1,1,1,有,有,7300000,...,0,0,1,0,0,1,1,0,2,966
2,86.0,全,透天厝,175.72,5,4,3,有,有,24500000,...,0,0,1,0,0,1,1,0,2,189
3,0.0,一層,公寓(5樓含以下無電梯),79.57,4,1,1,有,無,8500000,...,0,0,1,0,0,0,1,0,2,904


In [26]:
# 查看 "單價元平方公尺" 欄位的空值數量
null_count = df_merged['單價元平方公尺'].isnull().sum()

# 輸出結果
print("單價元平方公尺的空值數量：", null_count)
print("單價元平方公尺的空值比例：", null_count/58257)

單價元平方公尺的空值數量： 2615
單價元平方公尺的空值比例： 0.04488730967952349


In [27]:
# 移除 "單價元平方公尺" 欄位為空值的資料列
df_merged.dropna(subset=['單價元平方公尺'], inplace=True)

# 輸出結果
print("移除空值後的資料筆數：", len(df_merged))

移除空值後的資料筆數： 55642


In [28]:
# 移除不需要的欄位
df_merged = df_merged.drop(['土地移轉總面積平方公尺', '建物移轉總面積平方公尺', '總價元', '車位移轉總面積平方公尺',
               '備註', '主建物面積', '附屬建物面積', '陽台面積'], axis=1)

In [29]:
# 建立層次字串映射的字典
layer_mapping = {
    '地下一層': -1, '地下二層': -2, '地下三層': -3, '地下四層': -4, '地下五層': -5,
    '一層': 1, '二層': 2, '三層': 3, '四層': 4, '五層': 5,
    '六層': 6, '七層': 7, '八層': 8, '九層': 9, '十層': 10,
    '十一層': 11, '十二層': 12, '十三層': 13, '十四層': 14, '十五層': 15,
    '十六層': 16, '十七層': 17, '十八層': 18, '十九層': 19, '二十層': 20,
    '二十一層': 21, '二十二層': 22, '二十三層': 23, '二十四層': 24, '二十五層': 25, 
    '二十六層': 26, '二十七層': 27, '二十八層': 28, '二十九層': 29, '三十層': 30,
}

def extract_number(text):
    if isinstance(text, str):
        for number in sorted(layer_mapping.keys(), key=len, reverse=True):
            match = re.search(number, text)
            if match:
                return layer_mapping[number]
    return 0

# 將 '移轉層次' 欄位中的中文數字轉換為阿拉伯數字
df_merged['移轉層次-數字'] = df_merged['移轉層次'].apply(extract_number)


In [30]:
df_merged.drop('移轉層次', axis=1, inplace=True)

In [31]:
df_merged

,建物型態,建物現況格局-房,建物現況格局-廳,建物現況格局-衛,建物現況格局-隔間,有無管理組織,單價元平方公尺,車位類別,電梯,預售屋,...,鄉鎮市區_大安區,鄉鎮市區_文山區,鄉鎮市區_松山區,鄉鎮市區_萬華區,交易土地,交易建物,交易車位,交易距今,路段,移轉層次-數字
1,住宅大樓(11層含以上有電梯),1,1,1,有,有,178092,NaN,有,0,...,0,1,0,0,1,1,0,2,966,5
2,透天厝,5,4,3,有,有,139426,NaN,無,0,...,0,1,0,0,1,1,0,2,189,0
3,公寓(5樓含以下無電梯),4,1,1,有,無,106824,NaN,無,0,...,0,1,0,0,0,1,0,2,904,1
4,公寓(5樓含以下無電梯),3,2,2,有,無,110159,NaN,無,0,...,0,1,0,0,1,1,0,2,967,4
5,華廈(10層含以下有電梯),1,1,1,有,有,204499,NaN,有,0,...,0,1,0,0,1,1,0,5,965,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,其他,0,0,0,無,無,284434,NaN,NaN,1,...,0,0,1,0,5,1,0,0,119,4
642,華廈(10層含以下有電梯),2,1,1,有,無,387364,NaN,NaN,1,...,1,0,0,0,1,1,0,0,901,4
643,住宅大樓(11層含以上有電梯),3,2,2,有,無,354202,坡道平面,NaN,1,...,0,0,0,0,3,1,2,0,497,3
644,華廈(10層含以下有電梯),1,1,1,有,無,387161,NaN,NaN,1,...,1,0,0,0,1,1,0,0,1118,8


In [32]:
import numpy as np

# 填充空值
df_merged['電梯'] = df_merged['電梯'].fillna('未知')

# 條件判斷並填值
df_merged['電梯'] = np.where(
    (df_merged['電梯'] == '未知') & 
    (df_merged['建物型態'].isin(['住宅大樓(11層含以上有電梯)', '華廈(10層含以下有電梯)', '辦公商業大樓'])),
    1,
    0
)

In [33]:
# 把建物型態變成dummy variables
df_merged = pd.get_dummies(df_merged, columns=['建物型態'])
df_merged.head(3)

,建物現況格局-房,建物現況格局-廳,建物現況格局-衛,建物現況格局-隔間,有無管理組織,單價元平方公尺,車位類別,電梯,預售屋,建築完成距今,...,移轉層次-數字,建物型態_住宅大樓(11層含以上有電梯),建物型態_公寓(5樓含以下無電梯),建物型態_其他,建物型態_套房(1房1廳1衛),建物型態_店面(店鋪),建物型態_廠辦,建物型態_華廈(10層含以下有電梯),建物型態_辦公商業大樓,建物型態_透天厝
1,1,1,1,有,有,178092,NaN,0,0,26.0,...,5,1,0,0,0,0,0,0,0,0
2,5,4,3,有,有,139426,NaN,0,0,36.0,...,0,0,0,0,0,0,0,0,0,1
3,4,1,1,有,無,106824,NaN,0,0,54.0,...,1,0,1,0,0,0,0,0,0,0


In [34]:
df_merged['建物現況格局-隔間'] = df_merged['建物現況格局-隔間'].replace({'有': 1, '無': 0})
df_merged['有無管理組織'] = df_merged['有無管理組織'].replace({'有': 1, '無': 0})

In [35]:
df_merged

,建物現況格局-房,建物現況格局-廳,建物現況格局-衛,建物現況格局-隔間,有無管理組織,單價元平方公尺,車位類別,電梯,預售屋,建築完成距今,...,移轉層次-數字,建物型態_住宅大樓(11層含以上有電梯),建物型態_公寓(5樓含以下無電梯),建物型態_其他,建物型態_套房(1房1廳1衛),建物型態_店面(店鋪),建物型態_廠辦,建物型態_華廈(10層含以下有電梯),建物型態_辦公商業大樓,建物型態_透天厝
1,1,1,1,1,1,178092,NaN,0,0,26.000000,...,5,1,0,0,0,0,0,0,0,0
2,5,4,3,1,1,139426,NaN,0,0,36.000000,...,0,0,0,0,0,0,0,0,0,1
3,4,1,1,1,0,106824,NaN,0,0,54.000000,...,1,0,1,0,0,0,0,0,0,0
4,3,2,2,1,0,110159,NaN,0,0,14.508063,...,4,0,1,0,0,0,0,0,0,0
5,1,1,1,1,1,204499,NaN,0,0,2.000000,...,8,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,0,0,0,0,0,284434,NaN,0,1,0.000000,...,4,0,0,1,0,0,0,0,0,0
642,2,1,1,1,0,387364,NaN,1,1,0.000000,...,4,0,0,0,0,0,0,1,0,0
643,3,2,2,1,0,354202,坡道平面,1,1,0.000000,...,3,1,0,0,0,0,0,0,0,0
644,1,1,1,1,0,387161,NaN,1,1,0.000000,...,8,0,0,0,0,0,0,1,0,0


In [36]:
# 查看 "單價元平方公尺" 欄位的空值數量
null_count = df_merged['車位類別'].isnull().sum()

# 輸出結果
print("車位類別的空值數量：", null_count)
# print("單價元平方公尺的空值比例：", null_count/58257)

車位類別的空值數量： 34444


In [37]:
df_merged.drop('車位類別', axis=1, inplace=True)

In [38]:
df_merged

,建物現況格局-房,建物現況格局-廳,建物現況格局-衛,建物現況格局-隔間,有無管理組織,單價元平方公尺,電梯,預售屋,建築完成距今,鄉鎮市區_中山區,...,移轉層次-數字,建物型態_住宅大樓(11層含以上有電梯),建物型態_公寓(5樓含以下無電梯),建物型態_其他,建物型態_套房(1房1廳1衛),建物型態_店面(店鋪),建物型態_廠辦,建物型態_華廈(10層含以下有電梯),建物型態_辦公商業大樓,建物型態_透天厝
1,1,1,1,1,1,178092,0,0,26.000000,0,...,5,1,0,0,0,0,0,0,0,0
2,5,4,3,1,1,139426,0,0,36.000000,0,...,0,0,0,0,0,0,0,0,0,1
3,4,1,1,1,0,106824,0,0,54.000000,0,...,1,0,1,0,0,0,0,0,0,0
4,3,2,2,1,0,110159,0,0,14.508063,0,...,4,0,1,0,0,0,0,0,0,0
5,1,1,1,1,1,204499,0,0,2.000000,0,...,8,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,0,0,0,0,0,284434,0,1,0.000000,0,...,4,0,0,1,0,0,0,0,0,0
642,2,1,1,1,0,387364,1,1,0.000000,0,...,4,0,0,0,0,0,0,1,0,0
643,3,2,2,1,0,354202,1,1,0.000000,0,...,3,1,0,0,0,0,0,0,0,0
644,1,1,1,1,0,387161,1,1,0.000000,0,...,8,0,0,0,0,0,0,1,0,0


In [39]:
df_merged.to_csv('df_merged.csv', index=False)